初始化环境：
- 导包
- 加载环境变量
- 创建LLM对象

In [ ]:
from dotenv import load_dotenv
import os
from DeepSeekLLM import DeepSeekLLM

load_dotenv()
llm = DeepSeekLLM(api_key=os.getenv("deepseek_api_key"))

测试llm是否初始化成功

In [ ]:
response = llm.invoke("你好！")
print(response)

在 LangChain Chain 中使用

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["question"],
    template="回答以下问题：{question}"
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.invoke("你是谁？"))

提示词模版

In [12]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "你是一位翻译助手，将{input_language}翻译成{output_language}。"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='你是一位翻译助手，将English翻译成French。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]

输出解析器

In [9]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

LLMChain

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """你是一位助手，生成逗号分隔的列表。
用户会传入一个类别，你应该生成该类别中的5个对象，并以逗号分隔的列表返回。
只返回逗号分隔的列表，不要返回其他内容。"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=llm, # 使用DeepSeekLLM
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("国家")
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['中国', '美国', '日本', '德国', '巴西']